## Classification

This notebook demonstrates some of the classification tasks that can be accomplished using data retrieved from the HTRC API.

We will be using the Scikit Learn library to tackle these classification problems. Install the library using pip:
```
   pip install sklearn
```

For these examples, we will be gathering data using the advance search from the Hathi Trust library. To create your training set, determine a search query that will become your labels. Once you search, add the search results to a collection named after your label.

Once you have completed adding to your collection, go to "My Collections" under ____. There you will find a button called "Download Metadata".

Download the JSON file associated with your collection and place it in the local directory that you are working in.

In [1]:
import json
import os

jsonFiles = [file for file in os.listdir('.') if file.find('json') != -1]

txts = []
for file in jsonFiles:
    with open(file) as f:
        data = json.load(f)
        
    texts = data['gathers']
    ids = [text['htitem_id'] for text in texts]
    
    filename = data['title'] + '.txt'
    txts.append(filename)
    
    #write each id into txt file
    with open(filename, 'w') as f:
        for textid in ids:
            f.write(textid + '\n')

Once this step is complete, you can follow the instructions in the Setup notebook to load the data as needed.

## Task 1: Genre Classification

In this example, we'll be classifying texts into 3 different genres: Poetry, History, and Science Fiction. JSON files containing the metadata for 500 texts in each genre have been included.

In [11]:
from htrc_features import FeatureReader
import os

history_output = !htid2rsync --f history.txt | rsync -azv --files-from=- data.sharc.hathitrust.org::features/ history/

folder = 'history'
suffix = '.json.bz2'
filePaths = [path for path in history_output if path.endswith(suffix)]
history_paths = [os.path.join(folder, path) for path in filePaths]

poetry_output = !htid2rsync --f poetry.txt | rsync -azv --files-from=- data.sharc.hathitrust.org::features/ poetry/

folder = 'poetry'
suffix = '.json.bz2'
filePaths = [path for path in poetry_output if path.endswith(suffix)]
poetry_paths = [os.path.join(folder, path) for path in filePaths]

scifi_output = !htid2rsync --f scifi.txt | rsync -azv --files-from=- data.sharc.hathitrust.org::features/ scifi/

folder = 'scifi'
suffix = '.json.bz2'
filePaths = [path for path in scifi_output if path.endswith(suffix)]
scifi_paths = [os.path.join(folder, path) for path in filePaths]

print("Path extraction complete")

Path extraction complete


In [13]:
len(scifi_paths)

334